# Exercise 10 : Computing the static quark potential

### Load ensemble

In [1]:
import numpy as np
import os
from functools import partial
from re import split
from multiprocessing import Pool
import scipy.optimize as optimize
import matplotlib.pyplot as plt
%matplotlib inline

def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in split(r'(\d+)', string_)]

def load_ensemble(action, beta):
    #load ensemble files using action = "wilson_link_sum" or "improved_link_sum"
    ensemble = []
    directory = "gauge_configs/" + str(action) + "/beta_" + str(beta)  + "/"
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in sorted(files, key = natural_key):
            ensemble.append(np.load(os.path.abspath(os.path.join(root, name)), allow_pickle=True))
                        
    return ensemble

In [2]:
def link(lattice, coordinates, mu):
        
    n_points = lattice.shape[0]
        
    return lattice[coordinates[0] % n_points, coordinates[1] % n_points, 
                   coordinates[2] % n_points, coordinates[3] % n_points, mu, : , :]

def smeared_link(lattice, coordinates, mu, epsilon, u_0):
    res = np.zeros(3, dtype = 'complex128')
    
    coords_mu = coordinates[:]
    coords_mu[mu] += 1
    
    for rho in range(4):
        
        coords_rho = coordinates[:]
        coords_rho[rho] += 1
        
        coords_n_rho = coordinates[:]
        coords_n_rho -= 1
    
        coords_n_rho_mu = coords_mu[:]
        coords_n_rho_mu[rho] -= 1
        
        res += np.dot(np.dot(link(lattice, coordinates, rho), 
                             link(lattice, coords_rho, mu)),
                             link(lattice, coords_mu, rho).conj().T)
        
        res += np.dot(np.dot(link(lattice, coords_n_rho, rho).conj().T, 
                             link(lattice, coords_n_rho, mu)),
                             link(lattice, coords_n_rho_mu, rho))
        
    res = epsilon * res / u_0**2
    
    res += (1-8*epsilon) * link(lattice, coordinates, mu)
    
    return res

def smeared_n_link(lattice, coordinates, mu, epsilon, n, u_0):
    for i in range(n):
        res = 
        

        
        

I dont know how to code this up

In [3]:
def wilson_loop_2d(lattice, coordinates, spatial_direction1, spatial_extent1, temporal_extent):
    #each point has 3 temporal loops xt yt and zt, time is on exact same 
    #footing so doesnt really matter (euclidean isotropic)
    
    #starting bottom left, path of anti-quark in time ('static')
    bottom_res = link(lattice, coordinates, 0)
    bottom_coords_next = coordinates[:]
    
    #starting one in from top right as U^\dagger, path of quark in time ('static')
    top_coords = coordinates[:]
    top_coords[0] += temporal_extent-1
    top_coords[spatial_direction1] += spatial_extent1
    top_res = link(lattice, top_coords, 0).conj().T
    top_coords_next = top_coords[:]
    
    #links for sides of path
    left_coords = coordinates[:]
    left_coords[spatial_direction1] += spatial_extent1-1
    left_res = link(lattice, left_coords, spatial_direction1).conj().T
    left_coords_next = left_coords[:]
    
    right_coords = coordinates[:]
    right_coords[0] += temporal_extent
    right_res = link(lattice, right_coords, spatial_direction1)
    right_coords_next = right_coords[:]
    
    for i in range(temporal_extent-1):
    #calculate total link product from top and bottom paths
        
        bottom_coords_next[0] += 1
        bottom_link_next = link(lattice, bottom_coords_next, 0)
        bottom_res = np.dot(bottom_res, bottom_link_next)
    
        top_coords_next[0] -= 1
        top_link_next = link(lattice, top_coords_next, 0).conj().T
        top_res = np.dot(top_res, top_link_next)
    
    for j in range(spatial_extent1-1):
        #calculate total link product from right and left paths
        
        right_coords_next[spatial_direction1] += 1
        right_link_next = link(lattice, right_coords_next, spatial_direction1)
        right_res = np.dot(right_res, right_link_next)
        
        left_coords_next[spatial_direction1] -= 1
        left_link_next = link(lattice, left_coords_next, spatial_direction1).conj().T
        left_res = np.dot(left_res, left_link_next)
        
    bottom_right = np.dot(bottom_res, right_res)
    top_left = np.dot(top_res, left_res)
    total_loop = np.dot(bottom_right, top_left)
    
    return np.trace(total_loop).real / 3

def wilson_loop_3d(lattice, coordinates, spatial_direction1, spatial_extent1, 
                   spatial_direction2, spatial_extent2, temporal_extent):
    #loop which travels into a third dimension going side1 -> side2 -> side3 -> ... side6
    
    #side1, path of anti-quark in time ('static')
    side1_res = link(lattice, coordinates, 0)
    side1_coords_next = coordinates[:]
    
    #side2
    side2_coords = coordinates[:]
    side2_coords[0] += temporal_extent
    side2_res = link(lattice, side2_coords, spatial_direction1)
    side2_coords_next = side2_coords[:]
    
    #side3
    side3_coords = coordinates[:]
    side3_coords[0] += temporal_extent
    side3_coords[spatial_direction1] += spatial_extent1
    side3_res = link(lattice, side3_coords, spatial_direction2)
    side3_coords_next = side3_coords[:]
    
    #side4 with U^\dagger, path of quark in time ('static')
    side4_coords = coordinates[:]
    side4_coords[0] += temporal_extent-1
    side4_coords[spatial_direction1] += spatial_extent1
    side4_coords[spatial_direction2] += spatial_extent2
    side4_res = link(lattice, side4_coords, 0).conj().T
    side4_coords_next = side4_coords[:]
    
    #side5
    side5_coords = coordinates[:]
    side5_coords[spatial_direction1] += spatial_extent1
    side5_coords[spatial_direction2] += spatial_extent2 - 1
    side5_res = link(lattice, side5_coords, spatial_direction2).conj().T
    side5_coords_next = side5_coords[:]
    
    #side6
    side6_coords = coordinates[:]
    side6_coords[spatial_direction1] += spatial_extent1-1
    side6_res = link(lattice, side6_coords, spatial_direction1).conj().T
    side6_coords_next = side6_coords[:]
    
    for i in range(temporal_extent-1):
        #calculate total link products from temporal paths
        
        side1_coords_next[0] += 1
        side1_link_next = link(lattice, side1_coords_next, 0)
        side1_res = np.dot(side1_res, side1_link_next)
    
        side4_coords_next[0] -= 1
        side4_link_next = link(lattice, side4_coords_next, 0).conj().T
        side4_res = np.dot(side4_res, side4_link_next)

    
    for j in range(spatial_extent1-1):
        #calculate total link products from spatial_direction 1 paths
        
        side2_coords_next[spatial_direction1] += 1
        side2_link_next = link(lattice, side2_coords_next, spatial_direction1)
        side2_res = np.dot(side2_res, side2_link_next)
    
        side6_coords_next[spatial_direction1] -= 1
        side6_link_next = link(lattice, side6_coords_next, spatial_direction1).conj().T
        side6_res = np.dot(side6_res, side6_link_next)

    for k in range(spatial_extent2-1):
        #calculate total link products from spatial_direction 2 paths
        
        side3_coords_next[spatial_direction2] += 1
        side3_link_next = link(lattice, side3_coords_next, spatial_direction2)
        side3_res = np.dot(side3_res, side3_link_next)
    
        side5_coords_next[spatial_direction2] -= 1
        side5_link_next = link(lattice, side5_coords_next, spatial_direction2).conj().T
        side5_res = np.dot(side5_res, side5_link_next)

        
    total_loop = np.dot(np.dot(np.dot(np.dot(np.dot(side1_res, 
                                                    side2_res), 
                                                    side3_res), 
                                                    side4_res), 
                                                    side5_res), 
                                                    side6_res)
    
    return np.trace(total_loop).real / 3

def wilson_loop_4d(lattice, coordinates, spatial_direction1, spatial_extent1, 
                   spatial_direction2, spatial_extent2, spatial_direction3, spatial_extent3, temporal_extent):
    #loop which travels into a third dimension going side1 -> side2 -> side3 -> ... side8
    
    #side1, path of anti-quark in time ('static')
    side1_res = link(lattice, coordinates, 0)
    side1_coords_next = coordinates[:]
    
    #side2
    side2_coords = coordinates[:]
    side2_coords[0] += temporal_extent
    side2_res = link(lattice, side2_coords, spatial_direction1)
    side2_coords_next = side2_coords[:]
    
    #side3
    side3_coords = coordinates[:]
    side3_coords[0] += temporal_extent
    side3_coords[spatial_direction1] += spatial_extent1
    side3_res = link(lattice, side3_coords, spatial_direction2)
    side3_coords_next = side3_coords[:]
    
    #side4
    side4_coords = coordinates[:]
    side4_coords[0] += temporal_extent
    side4_coords[spatial_direction1] += spatial_extent1
    side4_coords[spatial_direction2] += spatial_extent2
    side4_res = link(lattice, side4_coords, spatial_direction3)
    side4_coords_next = side4_coords[:]    
    
    #side5 with U^\dagger, path of quark in time ('static')
    side5_coords = coordinates[:]
    side5_coords[0] += temporal_extent-1
    side5_coords[spatial_direction1] += spatial_extent1
    side5_coords[spatial_direction2] += spatial_extent2
    side5_coords[spatial_direction3] += spatial_extent3
    side5_res = link(lattice, side5_coords, 0).conj().T
    side5_coords_next = side5_coords[:]
    
    #side6
    side6_coords = coordinates[:]
    side6_coords[spatial_direction1] += spatial_extent1
    side6_coords[spatial_direction2] += spatial_extent2
    side6_coords[spatial_direction3] += spatial_extent3 - 1
    side6_res = link(lattice, side6_coords, spatial_direction3).conj().T
    side6_coords_next = side6_coords[:]
    
    #side7
    side7_coords = coordinates[:]
    side7_coords[spatial_direction1] += spatial_extent1
    side7_coords[spatial_direction2] += spatial_extent2 - 1
    side7_res = link(lattice, side7_coords, spatial_direction2).conj().T
    side7_coords_next = side7_coords[:]
    
    #side8
    side8_coords = coordinates[:]
    side8_coords[spatial_direction1] += spatial_extent1-1
    side8_res = link(lattice, side8_coords, spatial_direction1).conj().T
    side8_coords_next = side8_coords[:]
    
    for i in range(temporal_extent-1):
        #calculate total link products from temporal paths
        
        side1_coords_next[0] += 1
        side1_link_next = link(lattice, side1_coords_next, 0)
        side1_res = np.dot(side1_res, side1_link_next)
    
        side5_coords_next[0] -= 1
        side5_link_next = link(lattice, side5_coords_next, 0).conj().T
        side5_res = np.dot(side5_res, side5_link_next)

    
    for j in range(spatial_extent1-1):
        #calculate total link products from spatial_direction 1 paths
        
        side2_coords_next[spatial_direction1] += 1
        side2_link_next = link(lattice, side2_coords_next, spatial_direction1)
        side2_res = np.dot(side2_res, side2_link_next)
    
        side8_coords_next[spatial_direction1] -= 1
        side8_link_next = link(lattice, side8_coords_next, spatial_direction1).conj().T
        side8_res = np.dot(side8_res, side8_link_next)

    for k in range(spatial_extent2-1):
        #calculate total link products from spatial_direction 2 paths
        
        side3_coords_next[spatial_direction2] += 1
        side3_link_next = link(lattice, side3_coords_next, spatial_direction2)
        side3_res = np.dot(side3_res, side3_link_next)
    
        side7_coords_next[spatial_direction2] -= 1
        side7_link_next = link(lattice, side7_coords_next, spatial_direction2).conj().T
        side7_res = np.dot(side7_res, side7_link_next)
        
    for l in range(spatial_extent3-1):
        #calculate total link products from spatial_direction 3 paths
        
        side4_coords_next[spatial_direction3] += 1
        side4_link_next = link(lattice, side4_coords_next, spatial_direction3)
        side4_res = np.dot(side4_res, side4_link_next)
    
        side6_coords_next[spatial_direction3] -= 1
        side6_link_next = link(lattice, side6_coords_next, spatial_direction3).conj().T
        side6_res = np.dot(side6_res, side6_link_next)

        
    total_loop = np.dot(np.dot(np.dot(np.dot(np.dot(np.dot(np.dot(side1_res, 
                                                                  side2_res), 
                                                                  side3_res), 
                                                                  side4_res), 
                                                                  side5_res), 
                                                                  side6_res),
                                                                  side7_res),
                                                                  side8_res)
    
    return np.trace(total_loop).real / 3


In [4]:
def lattice_average_2d_loop(lattice, spatial_extent1, temporal_extent):
    dim = lattice.shape[0]
    res = []
    for t in range(dim):
        for x in range(dim):
            for y in range(dim):
                for z in range(dim):
                    for sd1 in range(1,4):
                        res.append(wilson_loop_2d(lattice, [t,x,y,z], spatial_direction1=sd1, 
                                              spatial_extent1=spatial_extent1, temporal_extent=temporal_extent))
                        
    return np.mean(res)

def lattice_average_3d_loop(lattice, spatial_extent1, spatial_extent2, temporal_extent):
    dim = lattice.shape[0]
    res = []
    for t in range(dim):
        for x in range(dim):
            for y in range(dim):
                for z in range(dim):
                    for sd1 in range(1,4):
                        for sd2 in range(1,4):
                            if sd1 != sd2:
                                res.append(wilson_loop_3d(lattice, [t,x,y,z], spatial_direction1=sd1, 
                                                          spatial_extent1=spatial_extent1, spatial_direction2=sd2, 
                                                          spatial_extent2=spatial_extent2, 
                                                          temporal_extent=temporal_extent))
                        
    return np.mean(res)

def lattice_average_4d_loop(lattice, spatial_extent1, spatial_extent2, spatial_extent3, temporal_extent):
    dim = lattice.shape[0]
    res = []
    for t in range(dim):
        for x in range(dim):
            for y in range(dim):
                for z in range(dim):
                    for sd1 in range(1,4):
                        for sd2 in range(1,4):
                            for sd3 in range(1,4):
                                if sd1 != sd2 and sd1 != sd3 and sd2 != sd3:
                                    res.append(wilson_loop_4d(lattice, [t,x,y,z], spatial_direction1=sd1, 
                                                              spatial_extent1=spatial_extent1, 
                                                              spatial_direction2=sd2, 
                                                              spatial_extent2=spatial_extent2, 
                                                              spatial_direction3=sd3, 
                                                              spatial_extent3=spatial_extent3, 
                                                              temporal_extent=temporal_extent))
                        
    return np.mean(res)


In [5]:
def loop_2d_time_array(lattice, spatial_extent1):
    dim = lattice.shape[0]
    loop_arr = []
    for temporal_extent in range(dim):
        val = lattice_average_2d_loop(lattice, spatial_extent1, temporal_extent)
        loop_arr.append(val)
        
    return loop_arr

def loop_3d_time_array(lattice, spatial_extent1, spatial_extent2):
    dim = lattice.shape[0]
    loop_arr = []
    for temporal_extent in range(dim):
        val = lattice_average_3d_loop(lattice, spatial_extent1, spatial_extent2, temporal_extent)
        loop_arr.append(val)
        
    return loop_arr

def loop_4d_time_array(lattice, spatial_extent1, spatial_extent2, spatial_extent3):
    dim = lattice.shape[0]
    loop_arr = []
    for temporal_extent in range(dim):
        val = lattice_average_4d_loop(lattice, spatial_extent1, spatial_extent2, spatial_extent3, temporal_extent)
        loop_arr.append(val)
        
    return loop_arr   

In [6]:
def gauge_array_2d(lattice_ensemble, spatial_extent1):
    p = Pool(5)
    func = partial(loop_2d_time_array, spatial_extent1 = spatial_extent1)
    loop_ensemble = np.array(list(p.map(func, lattice_ensemble)))
    p.terminate()
    
    return loop_ensemble, float(spatial_extent1)

def gauge_array_3d(lattice_ensemble, spatial_extent1, spatial_extent2):
    p = Pool(5)
    func = partial(loop_3d_time_array, spatial_extent1 = spatial_extent1, spatial_extent2 = spatial_extent2)
    loop_ensemble = np.array(list(p.map(func, lattice_ensemble)))
    p.terminate()
    
    return loop_ensemble, np.sqrt(spatial_extent1**2 + spatial_extent2**2)

def gauge_array_4d(lattice_ensemble, spatial_extent1, spatial_extent2, spatial_extent3):
    p = Pool(5)
    func = partial(loop_4d_time_array, spatial_extent1=spatial_extent1, spatial_extent2=spatial_extent2, 
                   spatial_extent3=spatial_extent3)
    loop_ensemble = np.array(list(p.map(func, lattice_ensemble)))
    p.terminate()
    
    return loop_ensemble, np.sqrt(spatial_extent1**2 + spatial_extent2**2 + spatial_extent3**2)


In [7]:
def jacknife(g):
    g_jacknife = []
    Ncf = len(g)
    for i in range(Ncf):
        b = np.delete(g,i,0)
        jknife_avg = np.mean(b, 0)
        g_jacknife.append(jknife_avg)
    return g_jacknife

def linear_func(x, c):
    return c


def analysis(ga, t1, t2):
    x0 = [0]
    n_configs = len(ga)
    
    jacknife_g = jacknife(ga)
    jacknife_g_shift = np.roll(jacknife_g, -1, axis = 1)
    V_eff = np.delete(np.log(np.divide(jacknife_g, jacknife_g_shift)), -1, 1)
    
    meanV = np.nanmean(V_eff, 0)
    stderrV = np.nanstd(V_eff,0)*np.sqrt(n_configs-1)
    time_fit = range(t1, t2)
    #Fitting
    fit_params = []
    for pot in V_eff:
        if np.isnan(pot[t1:t2]).any() == False:
            fit_param = optimize.curve_fit(linear_func, time_fit, 
                                           pot[t1:t2], np.array(x0), 
                                           sigma = stderrV[t1:t2])[0]
            fit_params.append(fit_param)
                
    fit_val = np.mean(fit_params)
    fit_val_err = np.std(fit_params)*np.sqrt(n_configs-1)
    
    return meanV, stderrV, fit_val, fit_val_err
                
def potential_data(action, beta, t1, t2):
    pot_dict = {}
    ensemble = load_ensemble(action, beta)
    
    #2D Loops
    for spatial_extent1 in range(1, 5):
        ga, dist = gauge_array_2d(ensemble, spatial_extent1)
        
        meanV, stderrV, fit_val, fit_val_err = analysis(ga, t1, t2)
        
        if dist not in pot_dict:
            pot_dict[dist] = [meanV, stderrV, fit_val, fit_val_err]
        elif dist in pot_dict:
            pot_dict[dist][0] = (pot_dict[dist][0] + meanV) / 2
            pot_dict[dist][1] = np.sqrt(pot_dict[dist][1]**2 + stderrV**2) / 2
            pot_dict[dist][2] = (pot_dict[dist][2] + fit_val) / 2
            pot_dict[dist][3] = np.sqrt(pot_dict[dist][3]**2 + fit_val_err**2) / 2
            
        print("Potential at dist: " + str(dist) + " calculated")

    
    #3D Loops
    for spatial_extent1 in range(1, 5):
        for spatial_extent2 in range(1, 4):
            ga, dist = gauge_array_3d(ensemble, spatial_extent1, spatial_extent2)
            meanV, stderrV, fit_val, fit_val_err = analysis(ga, t1, t2)
        
            if dist not in pot_dict:
                pot_dict[dist] = [meanV, stderrV, fit_val, fit_val_err]
            elif dist in pot_dict:
                pot_dict[dist][0] = (pot_dict[dist][0] + meanV) / 2
                pot_dict[dist][1] = np.sqrt(pot_dict[dist][1]**2 + stderrV**2) / 2
                pot_dict[dist][2] = (pot_dict[dist][2] + fit_val) / 2
                pot_dict[dist][3] = np.sqrt(pot_dict[dist][3]**2 + fit_val_err**2) / 2
                
            print("Potential at dist: " + str(dist) + " calculated")

            
    #4D Loops
    for spatial_extent1 in range(1, 4):
        for spatial_extent2 in range(1, 4):
            for spatial_extent3 in range(1, 3):
                ga, dist = gauge_array_4d(ensemble, spatial_extent1, spatial_extent2, spatial_extent3)
                meanV, stderrV, fit_val, fit_val_err = analysis(ga, t1, t2)
        
                if dist not in pot_dict:
                    pot_dict[dist] = [meanV, stderrV, fit_val, fit_val_err]
                elif dist in pot_dict:
                    pot_dict[dist][0] = (pot_dict[dist][0] + meanV) / 2
                    pot_dict[dist][1] = np.sqrt(pot_dict[dist][1]**2 + stderrV**2) / 2
                    pot_dict[dist][2] = (pot_dict[dist][2] + fit_val) / 2
                    pot_dict[dist][3] = np.sqrt(pot_dict[dist][3]**2 + fit_val_err**2) / 2
                    
                print("Potential at dist: " + str(dist) + " calculated")

    return pot_dict  

In [ ]:
def generate_plots(data, t1, t2):
    
    plot_num = len(data)
    fig, axs =  plt.subplots(nrows=plot_num, ncols=1, figsize=(6, 60), dpi=100)
    fig.subplots_adjust(hspace=0.3)
    distance_arr = sorted(data.keys())
    val_arr = []
    val_err_arr = []
    
    for i, key in enumerate(distance_arr):

        t = np.arange(0, 7)
        V_eff = data[key][0]
        V_eff_err = data[key][1]
        fit_val = data[key][2]
        fit_val_err = data[key][3]
        val_arr.append(fit_val)
        val_err_arr.append(fit_val_err)
        
        #Plotting
        axs[i].errorbar(t, V_eff, yerr=V_eff_err, fmt='o',  markersize=2, color='k', 
                        label = "Fit: y = " + str(round(fit_val,3)) + " error: " 
                        + str(round(fit_val_err,3)))
        axs[i].fill_between(t[t1:t2], np.ones(t[t1:t2].size)*fit_val-fit_val_err,
                         np.ones(t[t1:t2].size)*fit_val+fit_val_err,facecolor='blue',
                         alpha=0.5)
        axs[i].tick_params(bottom="off", top="off", left="off", right="off")
        axs[i].set_title("Static quark potential at distance " + str(round(key, 2))+"a")
        axs[i].set_xlim(0, 7)
        axs[i].set_ylim(0, 3)
        axs[i].set_xticks([0, 2, 4, 6, 8])
        axs[i].set_yticks([0, 0.5, 1, 1.5, 2, 2.5, 3])
        axs[i].legend()
        
    fig1 = plt.figure()
    ax = fig1.add_subplot(111)
    ax.errorbar(distance_arr, val_arr, yerr=val_err_arr, fmt='o',  markersize=2, color='k')
    ax.tick_params(bottom="off", top="off", left="off", right="off")
    ax.set_title("Static quark potential")
    ax.set_ylabel("aV(r)")
    ax.set_xlabel("a")
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 3)
    ax.set_xticks([0, 1, 2, 3, 4, 5])
    ax.set_yticks([0, 0.5, 1, 1.5, 2, 2.5, 3])
    
    
    plt.show()

In [ ]:
wilson_potential = potential_data("wilson_link_sum", 5.5, t1=2, t2=5)
improved_potential = potential_data("improved_link_sum", 1.719, t1=2, t2=5)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:22: RuntimeWarning: Mean of empty slice
/home/shaun/.local/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/shaun/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/shaun/.local/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/shaun/.local/lib/python3.5/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/shaun/.local/lib/python3.5/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe'

In [ ]:
generate_plots(wilson_potential, t1=, t2=5)

In [ ]:
generate_plots(improved_potential, t1=2, t2=5)